In [1]:
import cffi
from pynq import Overlay,PL,Xlnk
from fir import general_const
import timeit
import time
import numpy as np


#load bitstream
libfile = general_const.LIBRARY
bitfile = general_const.BITFILE

#init cffi (this must be before mem_init() for some reason?)
affi = cffi.FFI()
affi.cdef("void _p0_cpp_FIR_1_noasync(int *x, int w[85], int *ret, int datalen);")
alib = affi.dlopen(libfile)
    
xlnk = Xlnk()
fir_overlay = Overlay(bitfile)
if PL.bitfile_name != bitfile:
    fir_overlay.download()

#xlnk.xlnk_reset()
def mem_init(buflen):
    """ Allocate contiguous memory buffer
    """
    buf = xlnk.cma_alloc(buflen)
    return xlnk.cma_cast(buf, "int")

number=2000
n = 1000
dlen = n
din = mem_init(n*4)
win = mem_init(85)
dout = mem_init(n*4)

### Execution time: ABI v API

In [2]:
from fir import ffi, lib

# call ABI accelerator
def abiresp():
    alib._p0_cpp_FIR_1_noasync(din,win,dout,dlen)

# call API accelerator
def apiresp():
    lib._p0_cpp_FIR_1_noasync(din,win,dout,dlen)    


print("Running the benchmarks")
abi_time = timeit.timeit(abiresp,number=number)
api_time = timeit.timeit(apiresp,number=number)

print("Time taken by abi", number,"times",abi_time)
print("Time taken by api", number,"times",api_time)
print("API Speedup = %dx"%(abi_time/api_time))


Running the benchmarks
Time taken by abi 2000 times 1.907438557000205
Time taken by api 2000 times 1.9033871480005473
API Speedup = 1x
